## Cosas ha hacer: ##
1. Integracion: 
    - Obtención de datos
    - Explicación del dataset: OK
    - Resolver duplicidades e incosistencias.
    - Resolver problemas de codificacion (unidades de medida Km, m).
2. Preprocesamiento:
    - Outliers: OK.
    - Valores faltantes: OK.(Obtener por la mediana o por algun metodo REV).
    - Normalizar: No es necesario ya que con el tratamiento de los outliers los valores son manejables.
    - Discretizar
    - Selección/Reducción de variables.
3. Modelo: (Conjunto Entrenamiento, Test y Validación)
    - KNN.
    - Regresión lógistica. (Filtros ruido TOMEK LINKS, CVCF)
    - Arboles.
4. Evalución: Metodologias dividir datos entrenamiento, validacion y test
    - Matriz de confusión.
    - Clasificacion precision predictiva(Usando Grid).
    - Error cuadratico medio.
5. Interpretación y difusión:
    - Usos del modelo.



## Proyecto Final
- Yeray Aller Errea
- Javier Aranguren Ortiz
### Resumen
El objetivo de nuestro grupo va a ser utilizar las distintas herramientas que nos proporciona esta asignatura para identificar que tipo de trigo es.
### Dataset
En nuestro caso hemos seleccionado el siguiente dataset: [Seeds](https://www.kaggle.com/datasets/rwzhang/seeds-dataset)

El conjunto de datos Seeds contiene siete atributos geométricos de granos de trigo, y hay tres variedades de trigo. Las columnas en el archivo de datos representan los siguientes atributos:

- Área del grano **A**
- Perímetro del grano **P**
- Compacidad del grano **C** = 4 * pi * A / P^2
- Longitud del núcleo
- Ancho del núcleo
- Coeficiente de asimetría
- Longitud del surco del núcleo

### Desarrollo
![Descripción KDD](KDD.jpg)  
Para este proyeto vamos a utilizar la metodología KDD, que se divide en la siguientes etapas
1. Integracion y recopilacion.
2. Preprocesamiento.
3. Modelado.
4. Evalución.
5. Disfusiónn.


## INTEGRACIÓN Y RECOPILACIÓN ##
Para ese proyecto hemos seleccionado el dataset 

In [21]:
##Importamos todas las librerias necesarias para la práctica
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.base import TransformerMixin
from sklearn.metrics import accuracy_score

In [30]:
# Lectura del archivo csv
seeds = pd.read_csv('seeds_dataset.txt', delim_whitespace=True, header=None)
# Asignamos los nombres de las columnas
seeds.columns = ['area', 'perimetro', 'compacidad', 'longitud_nucleo', 'ancho_nucleo', 'coeficiente_asimetria', 'longitud_surco', 'class']
seeds

,area,perimetro,compacidad,longitud_nucleo,ancho_nucleo,coeficiente_asimetria,longitud_surco,class
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


Comprobamos si hay alguna fila que tenga la columna class vacia

In [23]:
if(seeds["class"].isnull().sum()==0):
    print("No hay clases sin valor")
else:
    print("Hay clases sin valor")

No hay clases sin valor


Generamos dos DataFrames: uno para la información de entrada (X) y otro para la de salida (y)

In [31]:
X = seeds.iloc[:, :-1]
y = seeds.iloc[:, -1]

A continuación vamos a dividir los ejemplos en train, validación y test utilizando Hold-out con estratificación y el valor 42 como semilla.

Vamos a dividir los conjuntos en un 10% test, 80% entrenamiento y el restante validacion.

In [32]:
X_resto, X_test, y_resto, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_resto, y_resto, test_size=0.2, stratify=y_resto, random_state=42)

Creamos la clase de detección y tratamiento de Outliers

In [33]:
class OutlierDetecion_treatment_IQR(TransformerMixin):

    # Constructor de la clase
    def __init__(self, k=1.5, columns=None):
        self.k = k
        self.columns = columns
    
    # Método fit
    def fit(self, X, y=None):
        # Transformamos X a DataFrame por si llega un array de Numpy (para compatibilidad en la Pipeline)
        X = pd.DataFrame(X)
        if self.columns == None:
            # Si no se determinan variables en el constructor si tratan todas
            self.columns = X.columns
            #self.stats = <RELLENAR>
            self.stats = X.describe()
        # Devolvemos el propio objeto modificado
        return self

    # Método transform
    def transform(self, X):
        # Transformamos X a DataFrame por si llega un array de Numpy (para compatibilidad en la Pipeline)
        X = pd.DataFrame(X)
        # Creamos una copia del DataFrame X para no perder los datos originales
        Xaux = X.copy()
        # Se calula el IQR de cada variable
        IQRs = self.stats.loc['75%'] - self.stats.loc['25%']
        # Se calculan los límites inferiores y superiores   
        limiteInf = self.stats.loc['25%'] - self.k * IQRs
        limiteSup = self.stats.loc['75%'] + self.k * IQRs
        # Se comprueba qué elementos están por encima y por debajo de dichos límites (máscaras de booleanos)  
        menores = (X < limiteInf)
        mayores = (X > limiteSup)
        # Se recorren las variables para detectar outliers y tratarlos (sustituir por la mediana de la variable)
        for c in self.columns:
            # obtenemos la lista de booleanos correspondientes a si los valores de los ejemplos son outliers o no para la variable c
            indices = np.logical_or(menores[c], mayores[c])
            # Si hay outliers
            if indices.any():
                # Los sustituimos por la mediana
                Xaux.loc[indices,c] = self.stats.loc['50%',c]
        # Se devuelve el DataFrame modificado
        return Xaux
    
    # Método para asignar los valores de los híper-parámetros y que, de este modo, 
        # podamos aplicar GridSearchCV sobre un objeto de esta clase
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    # Método para obtener los valores de los híper-parámetros que queramos del modelo (lo usa GridSearchCV al mostrar la mejor configuración)
    def get_params(self, deep=True):
        # Devolvemos los valores de los híper-parámetros del método de preparación de datos
        return {"k": self.k}

In [34]:
out_IQR = OutlierDetecion_treatment_IQR(k=3)
X_train_IQR = out_IQR.fit_transform(X_train)
if(X_train.equals(X_train_IQR)):
   print("No se han encontrado outliers")
else:
   print("Se han encontrado outliers y el dataframe ha sido modificado")

No se han encontrado outliers


Ahora vamos a utilizar diferentes tecnicas de clasificacion para evaluar sus rendimientos.
# KNN

In [35]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2, 1.5, 3]
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid.fit(X_train, y_train)

print("Mejores hiperparámetros:", grid.best_params_)
print("Exactitud en conjunto de validación:", grid.best_score_)

Mejores hiperparámetros: {'n_neighbors': 5, 'p': 1.5, 'weights': 'distance'}
Exactitud en conjunto de validación: 0.9539784946236558


# Regresion

In [ ]:
print("a")

# Arboles de decision